In [ ]:
# Import lib
import pandas as pd
import numpy as np

In [ ]:
# Read data
data = pd.read_csv("./Data/population_en_v2.csv")
data

# Khám phá dữ liệu

In [ ]:
col_dtypes = data.dtypes
col_dtypes

Các cột hầu như có kiểu dữ liệu đúng đắn, chẳng hạn như "Provinces/city" và "Region" là tên nên có kiểu dữ liệu object.  
Cột "Year" để dễ xử lý cho các phần sau nên không đổi thành kiểu DateTime.  
Các cột còn lại là số nên có kiểu số thực

# Câu hỏi: Dự đoán dân số trong vòng 5 năm kế tiếp.

Ý nghĩa: Đưa ra góc nhìn ngắn hạn để điều chỉnh chế độ kế hoạch hóa gia đình, dự trù cho nền kinh tế, dự đoán được các biến động xã hội có thể xảy ra khi dân số tăng vượt ngưỡng nào đó, từ đó đưa ra kế hoạch điều chỉnh kinh tế, xã hội (như quy hoạch hóa để xây dựng chung cư ở các thành phố lớn)

Các bước thực hiện:
1. Trích xuất thông tin dân số cho các tỉnh thành
2. Sử dụng mô hình thích hợp để dự đoán
3. Đưa ra thông tin dự đoán

In [ ]:
# unique provinces
provinces = data["Provinces/city"].unique()

Tạo ra tập test mẫu với TP HCM

In [ ]:
HCM_data = data[data["Provinces/city"]=='Ho Chi Minh']
x = HCM_data[['Year']]
y = HCM_data['Average population']
x_train = x[:8]
y_train = y[:8]
x_test = x[8:]
y_test = y[8:]

Sử dụng mô hình hồi quy tuyến tính cho dữ liệu trên

In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(x_train, y_train)

Trực quan hóa dữ liệu huấn luyện

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x_train, y_train, color="black")
plt.plot(x_train, reg.predict(x_train), color="blue", linewidth=3)

Sử dụng độ đo MSE

In [ ]:
from sklearn.metrics import mean_squared_error
y_pred = reg.predict(x_test)
mean_squared_error(y_pred, y_test)

Áp dụng cho toàn bộ 63 tỉnh thành

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
scores = []
res = pd.DataFrame(columns = ['Provinces/city','Average population', 'Year'])
for province in provinces:
    province_data = data[data["Provinces/city"]==province]
    x = province_data[['Year']]
    y = province_data['Average population']
    x_train = x[:8]
    y_train = y[:8]
    x_test = x[8:]
    y_test = y[8:]
    reg = linear_model.LinearRegression()
    reg.fit(x_train.values, y_train.values)
    y_pred = reg.predict(x_test.values)
    scores.append(mean_squared_error(y_pred, y_test))
    pred_x = np.arange(2021, 2026)
    pred_y = reg.predict(pred_x.reshape(-1, 1))
    pred_data = pd.DataFrame(data = zip(np.array([province]*5), pred_y, pred_x), columns = ['Provinces/city','Average population', 'Year'])
    res = pd.concat([res, pred_data])
res = res.reset_index()
res = res.drop('index', axis=1)
res

Đánh giá mô hình trên bằng MSE cho ra điểm số:

In [ ]:
np.array(scores).mean()

Dự đoán dân số 5 năm sau theo từng vùng:

In [ ]:
import seaborn as sns
ax = sns.lineplot(data=res, x='Year', y = 'Average population' , hue='Provinces/city', markers=True, dashes=False, style='Provinces/city')
h,l = ax.axes.get_legend_handles_labels()
ax.axes.legend_.remove()
ax.legend(h,l, ncol=3, loc=0, bbox_to_anchor=(2.1, 1))
ax.set_xticks(np.arange(2021, 2026))

## Decision Tree

In [ ]:
data.Region.unique()

Tiền xử lý cột **Region** bằng cách gán nhãn số từ 0 cho các lớp tương ứng. Ví dụ ở đây Hong river delta sẽ là số 1

In [ ]:
from sklearn import preprocessing
data1 = data.copy()
le = preprocessing.LabelEncoder()
le.fit(data1.Region)
data1.Region = le.transform(data1.Region)
data1

Sử dụng mô hình cây quyết định hồi quy cho từng tỉnh thành

In [ ]:
from sklearn import tree
scores = []
res = pd.DataFrame(columns = ['Provinces/city','Average population', 'Year'])
for province in provinces:
    province_data = data1[data1["Provinces/city"]==province]
    x = province_data[['Population density', 'Population grow ratio', 'Sex ratio', 'Region', 'Year']]
    y = province_data['Average population']
    x_train = x[:8]
    y_train = y[:8]
    x_test = x[8:]
    y_test = y[8:]
    clf = tree.DecisionTreeRegressor()
    clf = clf.fit(x_train.values, y_train.values)
    y_pred = clf.predict(x_test.values)
    scores.append(mean_squared_error(y_pred, y_test))

Tính điểm dựa trên trung bình cộng điểm của các tỉnh thành

In [ ]:
np.array(scores).mean()

Các tham số để fine-tune

In [ ]:
parameters={"splitter":["best","random"],
            "max_depth" : [1,3,5,7],
           "min_samples_leaf":[1,2,3,4,5],
           "min_weight_fraction_leaf":[0.1,0.2,0.3, 0.4],
           "max_features":["auto","log2","sqrt",None],
           "max_leaf_nodes":[None,10,20] }

Chay mô hình fine-tune

In [ ]:
from sklearn.model_selection import GridSearchCV
tuning_model=GridSearchCV(clf,param_grid=parameters,scoring='neg_mean_squared_error',cv=3,verbose=3)
tuning_model.fit(x,y)

Tham số tốt nhất là:

In [ ]:
tuning_model.best_params_

Chạy lại mô hình

In [ ]:
scores_array = []
res = pd.DataFrame(columns = ['Provinces/city','Average population', 'Year'])
for province in provinces:
    province_data = data1[data1["Provinces/city"]==province]
    x = province_data[['Year', 'Population density', 'Population grow ratio', 'Sex ratio', 'Region']]
    y = province_data['Average population']
    x_train = x[:8]
    y_train = y[:8]
    x_test = x[8:]
    y_test = y[8:]
    clf = tree.DecisionTreeRegressor(max_depth = 5, max_features = None,
                                    max_leaf_nodes = 20, min_samples_leaf = 1, 
                                    min_weight_fraction_leaf = 0.1, splitter = 'random')
    clf = clf.fit(x_train.values, y_train.values)
    y_pred = clf.predict(x_test.values)
    scores.append(mean_squared_error(y_pred, y_test))

Điểm số sau cùng của mô hình được tinh chỉnh

In [ ]:
np.array(scores).mean()